In [1]:
import ccxt
import pandas as pd
import matplotlib.pyplot as plt
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import numpy as np
import itertools

In [2]:
# spot exchanges we are onboarded
spot_exchanges = ['bitstamp','coinbasepro','liquid','bitfinex','binance','gemini',
                'bittrex','bitbank','huobipro','poloniex','kucoin2']   

#Crypto currencies we want to trade
Xpto= ['BTC','ETH','XRP','XMR','BCH','EOS','USDC','USDT','TRX','XLM','BSV','XBT','CSP']

#Fiat we want to trade 
Fiat=['USD','EUR','HKD','JPY','GBP','CHF']

#pairs from above
xpto_fiat = [xpto+'/'+ fiat for xpto in Xpto for fiat in Fiat] 
xpto_xpto=[p[0]+'/'+p[1] for p in itertools.permutations(Xpto,2)]
len(set(xpto_fiat+xpto_xpto))

234

In [3]:
#ccxt exchange objects dictionary {exch:ccxt.exch}
exch_dict={}
for x in spot_exchanges:
    exec('exch_dict[x]=ccxt.%s()' % (x))

In [4]:
#set of pairs on all my spot exchanges
all_pairs = set(sum(itertools.chain([*exch_dict[x].load_markets()] for x in exch_dict),[]))
len(all_pairs)

1655

In [5]:
pairs = list(set(xpto_fiat + xpto_xpto) & set(all_pairs))
pairs.sort()

In [6]:
len(pairs)

71

In [7]:
my_universe = pd.DataFrame(columns = pairs , index = spot_exchanges).T.reset_index()
for c in my_universe.columns[1:]:
      my_universe[c]= my_universe.apply(lambda x :'x' if x['index'] in exch_dict[c].load_markets().keys() else '0',axis=1 )
my_universe=my_universe.set_index('index')
my_universe
pair='BTC/USD'
my_universe

,bitstamp,coinbasepro,liquid,bitfinex,binance,gemini,bittrex,bitbank,huobipro,poloniex,kucoin2
index,,,,,,,,,,,
BCH/BTC,x,x,x,x,x,x,x,x,x,x,x
BCH/ETH,0,0,0,0,x,x,x,0,0,x,x
BCH/EUR,x,x,0,0,0,0,0,0,0,0,0
BCH/GBP,0,x,0,0,0,0,0,0,0,0,0
BCH/JPY,0,0,x,0,0,0,0,x,0,0,0
BCH/USD,x,x,x,x,0,x,x,0,0,0,0
BCH/USDC,0,0,0,0,x,0,0,0,0,x,0
BCH/USDT,0,0,0,0,x,0,x,0,x,x,x
BSV/BTC,0,0,0,x,x,0,x,0,x,x,x


In [8]:
def get_exchanges_for_pair(pair):
    return {x:exch_dict[x] for x in exch_dict if pair in list(exch_dict[x].load_markets().keys())}

In [9]:
def get_order_books(pair):
    '''pair is the pair string
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    ex= get_exchanges_for_pair(pair)
    if 'binance' not in ex:
        order_books = {key: value.fetch_order_book(pair,limit=2000,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in ex.items() }
    else:
        nobinance= {key:value for key, value in ex.items() if key != 'binance'}
        order_books = {key: value.fetch_order_book(pair,limit=2000,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in nobinance.items() }
        order_books['binance'] =  ex['binance'].fetch_order_book(pair,limit=1000)
    return order_books

In [10]:
def normalize_order_book(order_book,cutoff=.1,step=.001):
    '''order_book is a dictionary with keys bids asks timestamp datetime ...
    where bids is a list of list [[bid,bid_size]] and 
    asks is a list of list [[ask,ask_size]]
    this is returned by ccxt.'exchange'.fetch_order_book()
    returns a dataframe with columns [ask, ask_size, ask_size_$, cum_ask_size_$, bid_, bid_size, bid_size_$, cum_bid_size_$]
    and an index of shape np.linspace(1 - cutoff,1 + cutoff ,step =.001 ~ 10 bps)  
    '''
    try:
        rounding = int(np.ceil(-np.log(step)/np.log(10)))
        agg = True
    except:
        agg = False
    bid_side = pd.DataFrame(order_book['bids'],columns=['bid','bid_size'])
    bid_side['cum_bid_size'] = bid_side['bid_size'].cumsum()
    ask_side = pd.DataFrame(order_book['asks'],columns=['ask','ask_size'])
    ask_side['cum_ask_size'] = ask_side['ask_size'].cumsum()
    ref = (bid_side['bid'][0]+ask_side['ask'][0])/2
    bid_side['bid%'] = round(bid_side['bid']/ref,rounding) if agg else bid_side['bid']/ref
    ask_side['ask%'] = round(ask_side['ask']/ref,rounding) if agg else ask_side['ask']/ref
    bid_side = bid_side[bid_side['bid%']>=1-cutoff]
    ask_side = ask_side[ask_side['ask%']<=1+cutoff]
    bid_side['bid_size_$'] = bid_side['bid_size']*bid_side['bid']
    bid_side['cum_bid_size_$'] = bid_side['bid_size_$'].cumsum()
    ask_side['ask_size_$'] = ask_side['ask_size']*ask_side['ask']
    ask_side['cum_ask_size_$'] = ask_side['ask_size_$'].cumsum()
    normalized_bids = pd.DataFrame(bid_side.groupby('bid%',sort=False).mean()['bid'])
    normalized_bids.columns = ['bid']
    normalized_bids['bid_size'] = bid_side.groupby('bid%',sort=False).sum()['bid_size']
    normalized_bids['cum_bid_size'] = normalized_bids['bid_size'].cumsum()
    normalized_bids['bid_size_$'] = bid_side.groupby('bid%',sort=False).sum()['bid_size_$']
    normalized_bids['cum_bid_size_$'] = normalized_bids['bid_size_$'].cumsum()
    normalized_bids['average_bid_fill']=normalized_bids['cum_bid_size_$']/normalized_bids['cum_bid_size']
    normalized_asks = pd.DataFrame(ask_side.groupby('ask%',sort=False).mean()['ask'])
    normalized_asks.columns = ['ask']
    normalized_asks['ask_size'] = ask_side.groupby('ask%',sort=False).sum()['ask_size']
    normalized_asks['cum_ask_size'] = normalized_asks['ask_size'].cumsum()
    normalized_asks['ask_size_$'] = ask_side.groupby('ask%',sort=False).sum()['ask_size_$']
    normalized_asks['cum_ask_size_$'] = normalized_asks['ask_size_$'].cumsum()
    normalized_asks['average_ask_fill']=normalized_asks['cum_ask_size_$']/normalized_asks['cum_ask_size']
    book=pd.concat([normalized_asks,normalized_bids],sort=False)
    return book


In [11]:
def aggregate_order_books(dict_of_order_books):
    '''dict_of_order_books is a list of ccxt like order_books
        retuns a ccxt like dictionary order book sorted by prices 
    '''
    agg_dict_order_book = {}
    agg_dict_order_book['bids'] = (pd.DataFrame(list(itertools.chain(*[order_book['bids'] for order_book in dict_of_order_books.values()])))).sort_values(by=0,ascending=False).values.tolist()
    agg_dict_order_book['asks'] = (pd.DataFrame(list(itertools.chain(*[order_book['asks'] for order_book in dict_of_order_books.values()])))).sort_values(by=0,ascending=True).values.tolist()
    return agg_dict_order_book

In [12]:
def build_book(pair,exchanges,cutoff=.1,step=0.001):
    ''' gets order books aggreagtes them then normalizes
        returns a dataframe
    '''
    order_books = get_order_books(pair)
    return normalize_order_book(aggregate_order_books({key:order_books[key] for key in exchanges}),cutoff,step)

In [13]:
def plot_book(pair,exc,relative=True,cutoff=.1):
    ''' plots the order book as a v shape chart '''
    sel_book = build_book(pair,exc,cutoff)
    best_bid = round(sel_book['bid'].max(),3)
    best_ask = round(sel_book['ask'].min(),3)
    if relative:
        figure = go.Figure(data=[{'x':sel_book.index,'y':sel_book['cum_ask_size_$'],'name':'asks'}
                      ,{'x':sel_book.index,'y':sel_book['cum_bid_size_$'],'name':'bids'}],
     layout={'title':' / '.join(exc), 'xaxis':{'title':str(best_bid)+' / '+ str(best_ask)}})
    else:
        figure= go.Figure(data=[{'x':sel_book['ask'].fillna(0)+sel_book['bid'].fillna(0),'y':sel_book['cum_ask_size_$'],'name':'asks'}
                      ,{'x':sel_book['ask'].fillna(0)+sel_book['bid'].fillna(0),'y':sel_book['cum_bid_size_$'],'name':'bids'}],
     layout={'title':' / '.join(exc), 'xaxis':{'title':str(best_bid)+' / '+ str(best_ask)}})
    return figure

In [14]:
def order_fill(order_book_df, order_sizes,in_ccy=True):
    '''takes in an order book dataframe and an np.array of order sizes
        with size in currecncy by default else in coin
        returns an np.array of the purchase costs or the sale proceeds of an order
    '''
    average_fills = np.zeros(order_sizes.shape)
    mid=(order_book_df['ask'].min()+order_book_df['bid'].max())/2
    print(mid)
    if in_ccy:
        order_sizes=order_sizes/mid
    for i , order_size in enumerate(order_sizes):
        if order_size > 0:
            last_line=order_df[order_book_df['cum_ask_size']>order_size].iloc[0]
            #tail_fill = (last_line['cum_ask_size']-order_size)*last_line['ask']
            ccy_fill=last_line['cum_ask_size_$']+(order_size-last_line['cum_ask_size'])*last_line['ask']
            average_fill=ccy_fill/order_size
        elif order_size < 0:
            last_line=order_df[order_book_df['cum_bid_size'] > -order_size].iloc[0]
            #tail_fill = (last_line['cum_bid_size'] + order_size)*last_line['bid']
            ccy_fill=last_line['cum_bid_size_$']+(-order_size-last_line['cum_bid_size'])*last_line['bid']
            average_fill = -ccy_fill/order_size
        average_fills[i]=average_fill
    return average_fills

In [15]:
exchange_objects=get_exchanges_for_pair(pair)

In [16]:
exchanges = list(exchange_objects.keys())

In [17]:
exchanges

['bitstamp', 'coinbasepro', 'liquid', 'bitfinex', 'gemini', 'bittrex']

In [18]:
order_df = build_book(pair,exchanges,cutoff=.4,step=0.000000)
mid =.5*(order_df['bid'].max()+order_df['ask'].min())
mid

C:\Users\JeanEl-khoury\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

divide by zero encountered in log



3845.936525

In [19]:
len(order_df)

10176

In [20]:
order_fill(order_df,np.linspace(10,1e7,10))

3845.936525


array([3843.        , 3854.57597206, 3861.37343607, 3866.81345408,
       3872.845658  , 3882.18886809, 3891.48919892, 3901.98426735,
       3911.84320433, 3921.35543437])

In [21]:
order_fill(order_df,np.linspace(-10,-1e7,10))

3845.936525


array([3848.87305   , 3838.51891025, 3834.50594416, 3830.61455512,
       3826.37662201, 3820.71098464, 3814.12395964, 3807.30017637,
       3800.43572738, 3793.85334828])

In [22]:
order_df.columns

Index(['ask', 'ask_size', 'cum_ask_size', 'ask_size_$', 'cum_ask_size_$',
       'average_ask_fill', 'bid', 'bid_size', 'cum_bid_size', 'bid_size_$',
       'cum_bid_size_$', 'average_bid_fill'],
      dtype='object')

In [23]:
def plot_depth(order_book_df):
     mid = (order_book_df['bid'].max()+order_book_df['ask'].min())/2
     figure = go.Figure(data=[{'x':-order_book_df['cum_bid_size_$'],'y':order_book_df['average_bid_fill']/mid,'name':'bid depth'}
                      ,{'x':order_book_df['cum_ask_size_$'],'y':order_book_df['average_ask_fill']/mid,'name':'ask depth'}],
     layout={'title':' Market Depth'})
     return figure

In [24]:
iplot(plot_depth(order_df))

In [25]:
get_order_books(pair)

{'bitstamp': {'bids': [[3844.59, 1.0],
   [3844.58, 1.74833206],
   [3843.54, 0.59311739],
   [3841.8, 0.04],
   [3841.67, 3.8973],
   [3841.42, 0.5],
   [3840.68, 1.0],
   [3838.94, 8.38],
   [3838.81, 2.0],
   [3838.09, 0.029064],
   [3838.0, 0.04],
   [3837.81, 1.5],
   [3837.71, 4.516],
   [3837.1, 2.0],
   [3836.97, 7.4],
   [3835.06, 2.5],
   [3835.0, 70.0],
   [3834.77, 1.618],
   [3834.55, 8.5216],
   [3834.05, 0.00196276],
   [3833.3, 4.0],
   [3833.29, 0.4],
   [3832.72, 0.128],
   [3832.71, 7.833537],
   [3832.56, 0.35],
   [3831.42, 1.36694034],
   [3831.33, 2.376],
   [3830.86, 5.3807],
   [3830.0, 1.00206791],
   [3828.97, 0.00754],
   [3828.7, 3.8976],
   [3827.97, 6.95704487],
   [3827.45, 2.82447],
   [3826.85, 6.80777575],
   [3826.07, 6.12466865],
   [3824.99, 7.5],
   [3822.78, 4.21158913],
   [3822.36, 31.01299817],
   [3821.02, 0.0273003],
   [3821.0, 0.96180359],
   [3820.92, 7.9],
   [3819.65, 24.0],
   [3819.37, 1.452],
   [3818.3, 1.5],
   [3817.51, 7.8602506]

In [26]:
aggregate_order_books(get_order_books(pair))

{'bids': [[3849.82426, 0.22613],
  [3849.6634, 0.0141927],
  [3849.50174, 0.1],
  [3849.34906, 0.01],
  [3849.31322, 0.245],
  [3849.31313, 0.01],
  [3849.24146, 0.99107033],
  [3849.19638, 0.1],
  [3849.15461, 0.16005],
  [3848.89101, 0.01],
  [3848.57659, 0.01],
  [3848.47788, 0.02],
  [3848.28338, 0.151],
  [3848.15482, 0.014],
  [3848.15455, 0.01],
  [3848.01085, 0.1],
  [3847.66771, 0.9],
  [3847.30555, 0.01],
  [3846.38335, 0.05],
  [3846.12801, 0.5593],
  [3846.12792, 0.1],
  [3845.03, 1.0],
  [3844.85959, 0.21],
  [3844.21163, 0.8],
  [3844.15078, 0.003634],
  [3843.97825, 0.005169],
  [3843.43596, 0.16098],
  [3843.08244, 0.06],
  [3842.88, 0.003],
  [3842.68, 0.003],
  [3842.53227, 0.03],
  [3842.49016, 5.0],
  [3842.35264, 0.011183],
  [3841.77, 0.5],
  [3841.57127, 0.00608281],
  [3841.53535, 0.3],
  [3841.34153, 2.5],
  [3841.25693, 0.1095],
  [3840.98, 4.557],
  [3840.91564, 0.1071],
  [3840.68, 4.8987],
  [3840.66416, 0.05],
  [3840.59, 1.0],
  [3840.58962, 0.0263571],
 

In [27]:
import json
import time
import requests
import pandas as pd

class Sfox:
    def __init__(self, endpoint="api.sfox.com", version="v1"):
        self.endpoint = endpoint.lower()
        self.apiVersion = version.lower()

    def url_for(self, resource):
        return "https://" + self.endpoint + "/" + self.apiVersion + "/" + resource

    def __get(self, resource, apiKey=None):
        resource = resource.lower()
        auth = None
        if apiKey:
            auth = requests.auth.HTTPBasicAuth(apiKey, "")
        url = self.url_for(resource)
        return requests.get(url, auth=auth)

    def orderbook(self, market):
        market = market.lower()
        resource = "markets/orderbook/"+market
        res = self.__get(resource)
        return json.loads(res.text)

    def bestBuyPrice(self, quantity):
        resource = "offer/buy?amount="+str(quantity)
        res = self.__get(resource)
        data = json.loads(res.text)
        return data["price"]

    def bestSellPrice(self, quantity):
        resource = "offer/sell?amount="+str(quantity)
        res = self.__get(resource)
        data = json.loads(res.text)
        return data["price"]

In [28]:
sfox=Sfox()

In [29]:
ethusd=sfox.orderbook('ethusd')
ethusd

{'bids': [[136.31, 4, 'itbit'],
  [136.3, 10.39978764, 'itbit'],
  [136.29, 7.7156102, 'market1'],
  [136.28, 32, 'itbit'],
  [136.26, 0.28421255, 'market1'],
  [136.23, 47.24164858, 'itbit'],
  [136.22, 9.13367834, 'market1'],
  [136.21, 0.82, 'market1'],
  [136.2, 37.62, 'gemini'],
  [136.19, 4, 'market1'],
  [136.18, 2.16, 'market1'],
  [136.18, 4, 'bitstamp'],
  [136.17, 161.85400604, 'bitstamp'],
  [136.16, 163.07337356, 'bitstamp'],
  [136.15, 15, 'market1'],
  [136.15, 4, 'gemini'],
  [136.14, 202.15670107, 'bitstamp'],
  [136.12, 50, 'market1'],
  [136.11, 40.07135768, 'itbit'],
  [136.11, 50, 'gemini']],
 'asks': [[136.32, 9.421389, 'market1'],
  [136.33, 38.0487312, 'bitstamp'],
  [136.33, 10, 'itbit'],
  [136.34, 0.08, 'market1'],
  [136.41, 36.655, 'market1'],
  [136.41, 36.335, 'itbit'],
  [136.42, 9.43469651, 'market1'],
  [136.44, 4, 'itbit'],
  [136.46, 120.46753674, 'bitstamp'],
  [136.47, 15, 'market1'],
  [136.47, 51.3228, 'gemini'],
  [136.48, 15, 'market1'],
  [136

In [30]:
normalize_order_book(ethusd,cutoff=.1,step=.001)

AssertionError: 2 columns passed, passed data had 3 columns

In [ ]:
ethusd['bids']

In [ ]:
def normalize_order_book_2(order_book,cutoff=.1,step=.001):
    '''order_book is a dictionary with keys bids asks timestamp datetime ...
    where bids is a list of list [[bid,bid_size]] and 
    asks is a list of list [[ask,ask_size]]
    this is returned by ccxt.'exchange'.fetch_order_book()
    returns a dataframe with columns [ask, ask_size, ask_size_$, cum_ask_size_$, bid_, bid_size, bid_size_$, cum_bid_size_$]
    and an index of shape np.linspace(1 - cutoff,1 + cutoff ,step =.001 ~ 10 bps)  
    '''
    try:
        rounding = int(np.ceil(-np.log(step)/np.log(10)))
        agg = True
    except:
        agg = False
    bid_side = pd.DataFrame(order_book['bids'],columns=['bid','bid_size','bid_exc'])
    bid_side['cum_bid_size'] = bid_side['bid_size'].cumsum()
    ask_side = pd.DataFrame(order_book['asks'],columns=['ask','ask_size','ask_exc'])
    ask_side['cum_ask_size'] = ask_side['ask_size'].cumsum()
    ref = (bid_side['bid'][0]+ask_side['ask'][0])/2
    bid_side['bid%'] = round(bid_side['bid']/ref,rounding) if agg else bid_side['bid']/ref
    ask_side['ask%'] = round(ask_side['ask']/ref,rounding) if agg else ask_side['ask']/ref
    bid_side = bid_side[bid_side['bid%']>=1-cutoff]
    ask_side = ask_side[ask_side['ask%']<=1+cutoff]
    bid_side['bid_size_$'] = bid_side['bid_size']*bid_side['bid']
    bid_side['cum_bid_size_$'] = bid_side['bid_size_$'].cumsum()
    ask_side['ask_size_$'] = ask_side['ask_size']*ask_side['ask']
    ask_side['cum_ask_size_$'] = ask_side['ask_size_$'].cumsum()
    normalized_bids = pd.DataFrame(bid_side.groupby('bid%',sort=False).mean()['bid'])
    normalized_bids.columns = ['bid']
    normalized_bids['bid_size'] = bid_side.groupby('bid%',sort=False).sum()['bid_size']
    normalized_bids['cum_bid_size'] = normalized_bids['bid_size'].cumsum()
    normalized_bids['bid_size_$'] = bid_side.groupby('bid%',sort=False).sum()['bid_size_$']
    normalized_bids['cum_bid_size_$'] = normalized_bids['bid_size_$'].cumsum()
    normalized_bids['average_bid_fill']=normalized_bids['cum_bid_size_$']/normalized_bids['cum_bid_size']
    normalized_bids['bid_exc']=bid_side.groupby('bid%',sort=False)['bid_exc']
    normalized_bids['bid_exc']=normalized_bids['bid_exc'].apply(lambda x: (x[-1]))
    normalized_asks = pd.DataFrame(ask_side.groupby('ask%',sort=False).mean()['ask'])
    normalized_asks.columns = ['ask']
    normalized_asks['ask_size'] = ask_side.groupby('ask%',sort=False).sum()['ask_size']
    normalized_asks['cum_ask_size'] = normalized_asks['ask_size'].cumsum()
    normalized_asks['ask_size_$'] = ask_side.groupby('ask%',sort=False).sum()['ask_size_$']
    normalized_asks['cum_ask_size_$'] = normalized_asks['ask_size_$'].cumsum()
    normalized_asks['average_ask_fill']=normalized_asks['cum_ask_size_$']/normalized_asks['cum_ask_size']
    normalized_asks['ask_exc']=ask_side.groupby('ask%',sort=False)['ask_exc'].sum()
    #normalized_asks['ask_exc']=normalized_asks['ask_exc'].apply(lambda x: (x))
    book=pd.concat([normalized_asks,normalized_bids],sort=False)
    return book


In [ ]:
normalize_order_book_2(ethusd,cutoff=.1,step=0.00001)

In [ ]:
order_df['ask'].idxmax()

In [ ]:
ethusd['bids']

In [ ]:
ethusd={col:ethusd[col] for col in ['bids','asks']}

In [ ]:
ethusd.keys()

In [ ]:
def get_order_books_2(pair):
    '''pair is the pair string
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    ex= get_exchanges_for_pair(pair)
    if 'binance' not in ex:
        order_books = {key: value.fetch_order_book(pair,limit=5000,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in ex.items() }
    else:
        nobinance= {key:value for key, value in ex.items() if key != 'binance'}
        order_books = {key: value.fetch_order_book(pair,limit=5000,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in nobinance.items() }
        order_books['binance'] =  ex['binance'].fetch_order_book(pair,limit=1000)
    return order_books

In [ ]:
def get_order_books(pair):
    '''pair is the pair string
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    ex= get_exchanges_for_pair(pair)
    if 'binance' not in ex:
        order_books = {key: value.fetch_order_book(pair,limit=2000,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in ex.items() }
    else:
        nobinance= {key:value for key, value in ex.items() if key != 'binance'}
        order_books = {key: value.fetch_order_book(pair,limit=2000,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in nobinance.items() }
        order_books['binance'] =  ex['binance'].fetch_order_book(pair,limit=1000)
    return order_books

In [ ]:
%%timeit
get_order_books(pair)

In [ ]:
%%timeit
get_order_books_2(pair)

In [ ]:
bithumb=ccxt.bithumb()

In [ ]:
bithumb.load_markets()

In [ ]:
bithumb.fetch_order_book('BTC/KRW',limit=50,params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})

In [ ]:
4227000.0/1118

In [ ]:
binance=ccxt.binance()

In [ ]:
binance.fetch_order_book('BTC/USDT',limit=1000)

In [ ]:
binance.load_markets()

In [ ]:
binance.symbols

In [ ]:
huobi=ccxt.huobi()

In [ ]:
kucoin=ccxt.kucoin()

In [ ]:
kraken=ccxt.kraken()

In [ ]:
kraken.fees

In [ ]:
import datetime as dt

In [ ]:
snap=dt.datetime.strptime((dt.datetime.now().strftime("%Y-%m-%d  %H:%M:%S")),"%Y-%m-%d  %H:%M:%S")

In [ ]:
snap.strftime("%Y-%m-%d  %H:%M:%S")

In [ ]:
(dt.datetime.now()-snap).seconds

In [ ]:
bittrex=ccxt.bittrex()

In [ ]:
bittrex.load_markets()

In [ ]:
bittrex.fetch_order_book('BTC/USD')

In [ ]:
kucoin = ccxt.kucoin()